<a href="https://colab.research.google.com/github/simi12345/Sentiment-Analysis/blob/main/sample_SNAshop**.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nlp-id

In [ ]:
!pip install emoji

In [ ]:
import nltk
import random
from sklearn.model_selection import train_test_split
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re
import csv
from nltk.corpus import stopwords
import string
import emoji

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
with open('dataset/dataset_lazada.csv', 'r', encoding="utf8") as nodecsv: # Buka file
    csvreader = csv.reader(nodecsv) # membaca data
    # Menyusun data dalam list dan menghilangkan header data
    csv = [n for n in csvreader][1:]

In [ ]:
print(csv[:10])

[['ZM Zaskia Mecca - Jakarta Scarf Kerudung Segi Empat', 'Jilbab, warna deni m uku', 'negative', 'kesesuaian'], ['ZM Zaskia Mecca - Makassar Scarf Kerudung Segi Empat Wangi Water Splash', 'yg ini jga bgus bnget slalu puas blnja dsni dbanyakin promo ny y', 'positive', 'bagus'], ['ZM Zaskia Mecca - Cendana Scarf Kerudung Segi Empat Ultra Sonic - Romansa Khatulistiwa', 'Motifnya bagus banget. Sesuai real yang ada di gambar. bahan adem dan lembut. Terimakasih ZM. Next order lagi', 'positive', 'bagus'], ['ZM Zaskia Mecca - Izy Hijab Kerudung Bergo', 'bagus kerudungnyaa...sesuailah yaa....sama harganyaa.real pict pokonyyaaa.????', 'positive', 'bagus'], ['ZM Zaskia Mecca X Marsha Natika - Kyomi Pashmina', 'baRang sesuai pesanan cuma pengemasan Lama mgkn kRna bday juga x ea...Q Lebih suka ma yg kLavy,kyomi aga kaku bahan" y mgkn kaRna pLiskate...ini jiLbab k 7 daRi ZM sisa 3 masih otewe ceunah pokoke bagus2 semua Respon penjuaL Ramah', 'negative', 'late'], ['ZM Zaskia Mecca - Ezio Pants - Jeli

In [ ]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
regex_str = []
regex_str.append(emoticons_str)
regex_str.append(r'<[^>]+>')# HTML tags
regex_str.append(r'(?:@[\w_]+)')# @-mentions
regex_str.append(r'(?:&[\w_]+)')
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nlp_id.stopword import StopWord
stopword = StopWord()

In [ ]:
punctuation = list(string.punctuation)
stop = stopword.get_stopword() + punctuation + ['rt', 'via', '…','•','“']

In [ ]:
def tokenize(s):
    tokens = tokens_re.findall(s)
    return tokens

def cleanTweet(token,regex):
    terms_all = [emoji.demojize(term) for term in token if term.lower() not in stop and not regex.match(term)]
    return terms_all

In [ ]:
tokens = []
for c in csv:
    tokens.append(tokenize(c[1]))

print(tokens[:5])

[['Jilbab', ',', 'warna', 'deni', 'm', 'uku'], ['yg', 'ini', 'jga', 'bgus', 'bnget', 'slalu', 'puas', 'blnja', 'dsni', 'dbanyakin', 'promo', 'ny', 'y'], ['Motifnya', 'bagus', 'banget', '.', 'Sesuai', 'real', 'yang', 'ada', 'di', 'gambar', '.', 'bahan', 'adem', 'dan', 'lembut', '.', 'Terimakasih', 'ZM', '.', 'Next', 'order', 'lagi'], ['bagus', 'kerudungnyaa', '.', '.', '.', 'sesuailah', 'yaa', '.', '.', '.', '.', 'sama', 'harganyaa', '.', 'real', 'pict', 'pokonyyaaa', '.', '?', '?', '?', '?'], ['baRang', 'sesuai', 'pesanan', 'cuma', 'pengemasan', 'Lama', 'mgkn', 'kRna', 'bday', 'juga', 'x', 'ea', '.', '.', '.', 'Q', 'Lebih', 'suka', 'ma', 'yg', 'kLavy', ',', 'kyomi', 'aga', 'kaku', 'bahan', '"', 'y', 'mgkn', 'kaRna', 'pLiskate', '.', '.', '.', 'ini', 'jiLbab', 'k', '7', 'daRi', 'ZM', 'sisa', '3', 'masih', 'otewe', 'ceunah', 'pokoke', 'bagus', '2', 'semua', 'Respon', 'penjuaL', 'Ramah']]


In [ ]:
import operator

exclude_str = []
exclude_str.append(emoticons_str)
exclude_str.append(r'<[^>]+>')# HTML tags
exclude_str.append(r'(?:@[\w_]+)')# @-mentions
exclude_str.append(r'(?:&[\w_]+)')
exclude_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
exclude_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
exclude_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers

exclude_re = re.compile(r'('+'|'.join(exclude_str)+')', re.VERBOSE | re.IGNORECASE)

In [ ]:
all_word=[]
from collections import Counter
count_all = Counter()
for token in tokens:
    cleanToken = cleanTweet(token,exclude_re)
    terms_all = [term.lower() for term in cleanToken]
    count_all.update(terms_all)
    for w in terms_all:
        all_word.append(w)
print(count_all.most_common(100))

[('bagus', 58), ('zm', 29), ('lazada', 28), ('sesuai', 24), ('kurir', 21), ('suka', 19), ('bahan', 17), ('barang', 15), ('wangi', 15), ('seller', 15), ('warna', 14), ('bahannya', 14), ('adem', 13), ('pesanan', 12), ('gambar', 11), ('order', 11), ('ramah', 11), ('kasih', 11), ('harga', 11), ('ny', 10), ('pengiriman', 10), ('beli', 10), ('kerudung', 9), ('terima', 9), ('lg', 9), ('d', 9), ('terimakasih', 8), ('kurirnya', 8), ('kerudungnya', 8), ('lembut', 7), ('enak', 7), ('paketnya', 7), ('deh', 7), ('cocok', 7), ('n', 7), ('tp', 7), ('puas', 6), ('halus', 6), ('jd', 6), ('ga', 6), ('hijab', 6), ('warnanya', 6), ('gelap', 6), ('bgt', 6), ('next', 5), ('dipakai', 5), ('rapih', 5), ('thanks', 5), ('rapi', 5), ('realpict', 5), ('pesen', 5), ('coba', 5), ('semoga', 5), ('produk', 5), ('mksh', 5), ('beda', 5), ('cepat', 5), ('lumayan', 5), ('mantap', 5), ('ok', 5), ('udh', 5), ('jilbab', 4), ('slalu', 4), ('promo', 4), ('y', 4), ('real', 4), ('x', 4), ('to', 4), ('ciput', 4), ('abang', 4), (

In [ ]:
len(all_word)

1306

In [ ]:
from nlp_id.postag import PosTag
postagger = PosTag()

No model removed


/usr/local/lib/python3.7/dist-packages/nlp_id/postag.py:39: UserWarning: Downloading model ..
  warnings.warn("Downloading model ..")


In [ ]:
from nlp_id.lemmatizer import Lemmatizer
lemmatizer = Lemmatizer()

In [ ]:
all_stmword = [lemmatizer.lemmatize(w) for w in all_word]

In [ ]:
print(all_stmword[:100])

['jilbab', 'warna', 'den', 'm', 'uku', 'jga', 'bgus', 'bnget', 'slalu', 'puas', 'blnja', 'dsni', 'dbanyakin', 'promo', 'ny', 'y', 'motif', 'bagus', 'sesuai', 'real', 'gambar', 'bahan', 'adem', 'lembut', 'terimakasih', 'zm', 'next', 'order', 'bagus', 'kerudungnyaa', 'sesuai', 'yaa', 'harganyaa', 'real', 'pict', 'pokonyyaaa', 'barang', 'sesuai', 'pesan', 'kemas', 'mgkn', 'krna', 'bday', 'x', 'ea', 'q', 'suka', 'ma', 'klavy', 'kyomi', 'aga', 'kaku', 'bahan', 'y', 'mgkn', 'karna', 'pliskate', 'jilbab', 'k', 'zm', 'sisa', 'otewe', 'ceunah', 'pokoke', 'bagus', 'respon', 'jual', 'ramah', 'celana', 'keren', 'bahan', 'enak', 'pakai', 'corak', 'manis', 'serasi', 'dgn', 'tunik', 'zm', 'bagus', 'baju', 'thx', 'halus', 'bangettttttttttttt', 'jempol', 'suka', 'bangett', 'bikin', 'muka', 'cantik', 'paket', 'rapih', 'bungkus', 'kerudung', 'bagus', 'terima', 'kasih', 'thanks', 'to', 'lazada']


In [ ]:
pos = [postagger.get_pos_tag(w)[0] for w in all_stmword if len(postagger.get_pos_tag(w))>0]

In [ ]:
print(pos[:10])

[('jilbab', 'NN'), ('warna', 'NN'), ('den', 'NN'), ('m', 'NN'), ('uku', 'NN'), ('jga', 'NN'), ('bgus', 'NN'), ('bnget', 'NN'), ('slalu', 'NN'), ('puas', 'JJ')]


In [ ]:
allowed_word_types = ["ADV","JJ","ADJP","VB"]
allowed_words=[w[0] for w in pos if w[1] in allowed_word_types]

In [ ]:
print(allowed_words[:100])

['puas', 'dsni', 'bagus', 'sesuai', 'lembut', 'bagus', 'sesuai', 'sesuai', 'kyomi', 'kaku', 'bagus', 'jual', 'ramah', 'enak', 'pakai', 'manis', 'tunik', 'bagus', 'halus', 'bikin', 'cantik', 'rapih', 'bagus', 'terima', 'bagus', 'halus', 'rapi', 'sopan', 'sesuai', 'bagus', 'kirim', 'bagus', 'tpi', 'lepas', 'bagus', 'kirim', 'sale', 'ulang', 'kirim', 'sampe', 'ramah', 'coba', 'psti', 'coba', 'mantaap', 'lembut', 'wangi', 'cocok', 'aman', 'sampe', 'terima', 'lebar', 'beli', 'ori', 'biar', 'tmbh', 'beli', 'bergonya', 'tipis', 'bagus', 'enak', 'dipke', 'wangi', 'terssa', 'rapih', 'enak', 'pakai', 'sesuai', 'bagus', 'rapi', 'bagus', 'bagus', 'bagus', 'bagus', 'murah', 'sale', 'tipis', 'licin', 'tebal', 'beda', 'beda', 'kirim', 'cepat', 'sesuai', 'bagus', 'halus', 'beli', 'kecewa', 'wangi', 'ramah', 'bagus', 'bagus', 'sesuai', 'ramah', 'sopan', 'makasiiiih', 'puas', 'rejeki', 'disni', 'hijabny']


In [ ]:
#Menyusun distribusi kata berdasarkan kemunculannya
allwords = nltk.FreqDist(allowed_words)

#Mengambil 5000 pertama dari distribusi kata berdasarkan frekwensi kemunculannya
tuple_features = list(allwords.most_common())[:1000]
word_features = [k for (k,v) in tuple_features]
#Memeriksa word_features
print(word_features[:100])

['bagus', 'sesuai', 'wangi', 'terima', 'kirim', 'ramah', 'beli', 'puas', 'lembut', 'enak', 'cocok', 'tp', 'pakai', 'halus', 'coba', 'gelap', 'rapih', 'rapi', 'beda', 'cepat', 'lumayan', 'mantap', 'udh', 'nyaman', 'mksih', 'kyomi', 'sampe', 'tipis', 'kecewa', 'mudah', 'mecca', 'lgi', 'dsni', 'tunik', 'cantik', 'sopan', 'sale', 'ori', 'licin', 'hijabny', 'lagiiii', 'langsung', 'atur', 'neng', 'ragu', 'tunggu', 'serum', 'muda', 'dipake', 'terracota', 'kaku', 'jual', 'manis', 'bikin', 'tpi', 'lepas', 'ulang', 'psti', 'mantaap', 'aman', 'lebar', 'biar', 'tmbh', 'bergonya', 'dipke', 'terssa', 'murah', 'tebal', 'makasiiiih', 'rejeki', 'disni', 'i', 'dic', 'ssuai', 'repot', 'klavi', 'hana', 'terang', 'dateng', 'khas', 'gampang', 'dipakenya', 'antar', 'diksh', 'tetep', 'memuaskn', 'mngcewakan', 'lucu', 'kena', 'asli', 'ingsallah', 'lengkap', 'bicara', 'mentawai', 'makasiii', 'bru', 'kli', 'tulisnnya', 'dikit', 'mkasih']


In [ ]:
document=[]
for w in csv:
  document.append((w[1],w[2]))

In [ ]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [ ]:
nltk.download('punkt')
featuresets = [(find_features(rev), category) for (rev, category) in document]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print(featuresets[:5])

[({'bagus': False, 'sesuai': False, 'wangi': False, 'terima': False, 'kirim': False, 'ramah': False, 'beli': False, 'puas': False, 'lembut': False, 'enak': False, 'cocok': False, 'tp': False, 'pakai': False, 'halus': False, 'coba': False, 'gelap': False, 'rapih': False, 'rapi': False, 'beda': False, 'cepat': False, 'lumayan': False, 'mantap': False, 'udh': False, 'nyaman': False, 'mksih': False, 'kyomi': False, 'sampe': False, 'tipis': False, 'kecewa': False, 'mudah': False, 'mecca': False, 'lgi': False, 'dsni': False, 'tunik': False, 'cantik': False, 'sopan': False, 'sale': False, 'ori': False, 'licin': False, 'hijabny': False, 'lagiiii': False, 'langsung': False, 'atur': False, 'neng': False, 'ragu': False, 'tunggu': False, 'serum': False, 'muda': False, 'dipake': False, 'terracota': False, 'kaku': False, 'jual': False, 'manis': False, 'bikin': False, 'tpi': False, 'lepas': False, 'ulang': False, 'psti': False, 'mantaap': False, 'aman': False, 'lebar': False, 'biar': False, 'tmbh': F

In [ ]:
# menyusun data training dan testing
trainsize = round(len(featuresets)*0.7)

random.shuffle(featuresets)
training_set = featuresets[:trainsize]
testing_set = featuresets[trainsize:]


In [ ]:
print(*testing_set[:5], sep = "\n")

({'bagus': False, 'sesuai': False, 'wangi': True, 'terima': False, 'kirim': False, 'ramah': False, 'beli': False, 'puas': True, 'lembut': True, 'enak': False, 'cocok': False, 'tp': False, 'pakai': False, 'halus': False, 'coba': False, 'gelap': False, 'rapih': False, 'rapi': False, 'beda': False, 'cepat': False, 'lumayan': False, 'mantap': False, 'udh': False, 'nyaman': False, 'mksih': False, 'kyomi': False, 'sampe': False, 'tipis': False, 'kecewa': False, 'mudah': True, 'mecca': False, 'lgi': False, 'dsni': False, 'tunik': False, 'cantik': False, 'sopan': False, 'sale': False, 'ori': False, 'licin': False, 'hijabny': False, 'lagiiii': False, 'langsung': False, 'atur': False, 'neng': False, 'ragu': False, 'tunggu': False, 'serum': False, 'muda': False, 'dipake': False, 'terracota': False, 'kaku': False, 'jual': False, 'manis': False, 'bikin': False, 'tpi': False, 'lepas': False, 'ulang': False, 'psti': False, 'mantaap': False, 'aman': False, 'lebar': False, 'biar': False, 'tmbh': False,

In [ ]:
print('\n'.join(map(str, training_set[:5])))

({'bagus': False, 'sesuai': False, 'wangi': False, 'terima': False, 'kirim': False, 'ramah': False, 'beli': False, 'puas': False, 'lembut': False, 'enak': False, 'cocok': False, 'tp': False, 'pakai': False, 'halus': False, 'coba': False, 'gelap': False, 'rapih': False, 'rapi': False, 'beda': False, 'cepat': False, 'lumayan': False, 'mantap': False, 'udh': False, 'nyaman': False, 'mksih': False, 'kyomi': False, 'sampe': False, 'tipis': False, 'kecewa': False, 'mudah': False, 'mecca': False, 'lgi': False, 'dsni': False, 'tunik': False, 'cantik': False, 'sopan': False, 'sale': False, 'ori': False, 'licin': False, 'hijabny': False, 'lagiiii': False, 'langsung': False, 'atur': False, 'neng': False, 'ragu': False, 'tunggu': False, 'serum': False, 'muda': False, 'dipake': False, 'terracota': False, 'kaku': False, 'jual': False, 'manis': False, 'bikin': False, 'tpi': False, 'lepas': False, 'ulang': False, 'psti': False, 'mantaap': False, 'aman': False, 'lebar': False, 'biar': False, 'tmbh': Fa

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(25)

Classifier accuracy percent: 88.09523809523809
Most Informative Features
                      tp = True           negati : positi =      8.5 : 1.0
                   gelap = True           negati : positi =      6.1 : 1.0
                  nyaman = True           negati : positi =      6.1 : 1.0
                 lumayan = True           negati : positi =      3.6 : 1.0
                   bagus = True           positi : negati =      2.3 : 1.0
                    enak = True           negati : positi =      2.0 : 1.0
                   bagus = False          negati : positi =      1.4 : 1.0
                  sesuai = True           positi : negati =      1.4 : 1.0
                      tp = False          positi : negati =      1.3 : 1.0
                   ramah = True           negati : positi =      1.2 : 1.0
                  terima = True           negati : positi =      1.2 : 1.0
                   gelap = False          positi : negati =      1.2 : 1.0
                    beda = 

In [ ]:
from nltk.metrics.scores import (precision, recall)
import collections

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)

for i, (feats, label) in enumerate(testing_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)

print('Recall:', recall(refsets['positive'], testsets['positive']))
print('Precision:', precision(refsets['positive'], testsets['positive']))

Recall: 0.9
Precision: 0.972972972972973


In [ ]:
#menyimpan model yang telah ditrainning ke file sav
filename = 'nb_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [ ]:
#load model classifier nb dari file
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
#melakukan prediksi data menggunakan model classifier nb yang telah diload
lsPrediksi = []
for (rev, category) in document[trainsize:]:
    result=loaded_model.classify(find_features(rev))
    lsPrediksi.append([rev, result, category])

In [ ]:
print(lsPrediksi)

[['paket yang d tunggu tunggu Alhamdullah datang jga saya seneng banget sesuai gambar..serum nya bagus banget enak d pake nya muda2han cocok nanti bisa order lagi ..makasih laZada makasih ZM...', 'positive', 'positive'], ['kerudungnya bagus tp ada nodanya, sebaiknya lebih teliti lagi pengecekannya sebelum di packing', 'positive', 'positive'], ['Alhamdulillah paketnya sudah datang dan rapih bungkusnya pula, kerudung nya bagus, terima kasih banyak. thanks to lazada dan kurirnya juga....', 'positive', 'positive'], ['terimakasih kerudungnya bagus', 'positive', 'positive'], ['alhamdulillah barang sdh diterima,sesuai pesanan,kurir cukup ramah,pesanan cpt sampai....pokokny puas dh,trimqkasih lazada', 'positive', 'positive'], ['Rekomended seller dah. Bagus dah. Mantap dah.', 'positive', 'positive'], ['pengen beli lagiiii dan laagiiii, oke BGT', 'positive', 'positive'], ['baguuussss sekali kakak baguuss sukak pengiriman lama karna overload', 'positive', 'negative'], ["baRu kasih uLasan skRg pdh

In [ ]:
len(lsPrediksi)

42

In [ ]:
print(*lsPrediksi, sep = "\n")

['paket yang d tunggu tunggu Alhamdullah datang jga saya seneng banget sesuai gambar..serum nya bagus banget enak d pake nya muda2han cocok nanti bisa order lagi ..makasih laZada makasih ZM...', 'positive', 'positive']
['kerudungnya bagus tp ada nodanya, sebaiknya lebih teliti lagi pengecekannya sebelum di packing', 'positive', 'positive']
['Alhamdulillah paketnya sudah datang dan rapih bungkusnya pula, kerudung nya bagus, terima kasih banyak. thanks to lazada dan kurirnya juga....', 'positive', 'positive']
['terimakasih kerudungnya bagus', 'positive', 'positive']
['alhamdulillah barang sdh diterima,sesuai pesanan,kurir cukup ramah,pesanan cpt sampai....pokokny puas dh,trimqkasih lazada', 'positive', 'positive']
['Rekomended seller dah. Bagus dah. Mantap dah.', 'positive', 'positive']
['pengen beli lagiiii dan laagiiii, oke BGT', 'positive', 'positive']
['baguuussss sekali kakak baguuss sukak pengiriman lama karna overload', 'positive', 'negative']
["baRu kasih uLasan skRg pdhL nyampe 

In [ ]:
print(lsPrediksi[1][1])

positive


In [ ]:
if (lsPrediksi[1][1] == "negative"):
  print(lsPrediksi, sep = "\n")


In [ ]:
act=[]
pred=[]

for i, (feats, label) in enumerate(testing_set):
    observed = classifier.classify(feats)
    act.append(label)
    pred.append(observed)

In [ ]:
from nltk.metrics import ConfusionMatrix

In [ ]:
matrix = ConfusionMatrix(act,pred)

In [ ]:
print(matrix)

         |  n  p |
         |  e  o |
         |  g  s |
         |  a  i |
         |  t  t |
         |  i  i |
         |  v  v |
         |  e  e |
---------+-------+
negative | <1> 1 |
positive |  4<36>|
---------+-------+
(row = reference; col = test)



In [ ]:
classifier2 = nltk.classify.SklearnClassifier(LinearSVC())
classifier2.train(training_set)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0))>

In [ ]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier2, testing_set))*100)

Classifier accuracy percent: 85.71428571428571


In [ ]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, shuffle=False, random_state=None)

In [ ]:
for traincv, testcv in cv.split(featuresets):
    classifier = nltk.NaiveBayesClassifier.train(featuresets[traincv[0]:traincv[len(traincv)-1]])
    print ('accuracy: %.3f' % nltk.classify.util.accuracy(classifier, featuresets[testcv[0]:testcv[len(testcv)-1]]))

accuracy: 0.889
accuracy: 0.963
accuracy: 0.852
accuracy: 0.852
accuracy: 0.808


In [ ]:
from nltk.metrics.scores import (precision, recall)
import collections

In [ ]:
k=1
for traincv, testcv in cv.split(featuresets):
    training_sets = featuresets[traincv[0]:traincv[len(traincv)-1]]
    testing_sets = featuresets[testcv[0]:testcv[len(testcv)-1]]
    classifier = nltk.NaiveBayesClassifier.train(training_sets)
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)

    for i, (feats, label) in enumerate(testing_sets):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)
    print ("Fold - "+str(k))
    print('Recall:', recall(refsets['positive'], testsets['positive']))
    print('Precision:', precision(refsets['positive'], testsets['positive']))
    k+=1


Fold - 1
Recall: 0.9166666666666666
Precision: 0.9565217391304348
Fold - 2
Recall: 1.0
Precision: 0.9615384615384616
Fold - 3
Recall: 1.0
Precision: 0.8461538461538461
Fold - 4
Recall: 1.0
Precision: 0.8518518518518519
Fold - 5
Recall: 0.84
Precision: 0.9545454545454546
